In [1]:
import numpy as np

np.random.seed(13)
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from stockstats import StockDataFrame 
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
import dataPreparation
param = {
    'q': 'AAPL', # Stock symbol (ex: "AAPL")
    'i': "3600", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "NASD", # INDEXNASDAQStock exchange symbol on which stock is traded (ex: "NASD")
    'p': "2Y" # Period (Ex: "1Y" = 1 year)
    }
   
# link to doc http://www.networkerror.org/component/content/article/1-technical-wootness/44-googles-undocumented-finance-api.html
# get price data (return pandas dataframe) 1465948800
priceData = get_price_data(param)
#print(priceData)
stock = StockDataFrame.retype(priceData)
#print(stock)
string=['open_-6_d','rsi_3','rsi_6','rsi_12']

#['macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6''close_10_sma''tr''middle_{}_sma'
#df['dma'] = df['close_10_sma'] - df['close_50_sma'] pdi'] = cls._get_pdi(df, 14)df['mdi'] = cls._get_mdi(df, 14)
#df['dx'] = cls._get_dx(df, 14) df['adx'] = df['dx_6_ema'] df['adxr'] = df['adx_6_ema']]
#



stock.get(string)
price=stock
print(price)

/home/andrea/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/andrea/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


                         open      high       low     close    volume  \
2016-05-02 22:00:00   93.9650   94.0800   92.5500   92.5600   7281030   
2016-05-02 23:00:00   92.5700   93.4700   92.4000   92.7100   9191799   
2016-05-03 00:00:00   92.7000   93.1800   92.6900   92.9202   4437184   
2016-05-03 01:00:00   92.9200   92.9900   92.6100   92.7800   3772353   
2016-05-03 02:00:00   92.7899   93.0732   92.6700   92.7599   3878751   
2016-05-03 03:00:00   92.7600   93.6400   92.6800   93.6200   5284737   
2016-05-03 04:00:00   93.6200   93.9400   93.3500   93.6500  10132595   
2016-05-03 22:00:00   94.2000   94.4900   93.7500   93.9800   8772314   
2016-05-03 23:00:00   93.9750   94.2800   93.6800   94.0000   8441792   
2016-05-04 00:00:00   94.0000   94.8800   93.8500   94.6100   7296309   
2016-05-04 01:00:00   94.6100   95.0800   94.3701   95.0600   5892129   
2016-05-04 02:00:00   95.0600   95.7400   94.9600   95.3600   7099912   
2016-05-04 03:00:00   95.3700   95.4500   95.0900  

In [2]:




xtemp=price[string]

xtemp=np.nan_to_num(np.asarray(xtemp, dtype=float))
xtemp=normalize(xtemp,axis=0,norm='max')
print(xtemp)
accintime=[]
trendwindowtime=[2,6,12,30]
for t in trendwindowtime:
    #1
#label because of the maket and append values without data
#simo theroy past trend
    x=[]
    y=[]
    print('============================================================')
    print('============================================================')
    print('Working on window:',t)
    print(len(xtemp))
    ##QUI C E L'UNICO APPUNTO GUARDA SE CON +1 CAMBIA
    for i in range(0,len(price)-t-1):
        y.append(np.sign(price.iloc[i+t+1]['close']-price.iloc[i+1]['open']))
        x.append(xtemp[i])
    y=np.array(y)
    x=np.array(x)
    #Split betwwen train-validation and test
    #Faccio random permuatation cosi e come se lo facessi su 5 dataset completamentediversi
    
    
    permindex=range(0,len(x))
    #permindex=np.random.permutation(permindex)
    train=0.8
    nt=math.ceil(len(x)*train)
    trainvalindex=permindex[0:nt]
    testindex=permindex[nt:]
    x_tv=[]
    y_tv=[]
    x_test=[]
    y_test=[]
    x_tv=x[trainvalindex]
    y_tv=y[trainvalindex]
    x_test=x[testindex]
    y_test=y[testindex]

    #Fairly sampling the tv
    posindex=np.where( y_tv > 0 )
    negindex=np.where( y_tv < 0 )

    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    y_tvnew=np.concatenate((y_tv[posindex[0][0:nindex]],y_tv[negindex[0][0:nindex]]))
    x_tvnew=np.concatenate((x_tv[posindex[0][0:nindex]],x_tv[negindex[0][0:nindex]]))

    #Fairly sampling the test 50% 50%
    posindex=np.where( y_test > 0 )
    negindex=np.where( y_test < 0 )

    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
    x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))
    
    
    nfold=10
    kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


    cspace=np.logspace(-2,4,5)
    gspace=np.logspace(-4,4,5)
    bestsvm=None
    maxacc=0
    cvacc=0
    maxg=0
    maxc=0
    print('Model Selection...')
    #model selection
    for c in cspace:
        #print()
        for g in gspace:
            cvacc=0
            #crossvalidation
            for train_index, test_index in kf.split(x_tvnew):
                x_train, x_val = x_tvnew[train_index], x_tvnew[test_index]
                y_train, y_val = y_tvnew[train_index], y_tvnew[test_index]
                rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
                rbf_svm.fit(x_train,y_train)
                accuracy=rbf_svm.score(x_val,y_val)
                cvacc=cvacc+accuracy/nfold
            #print('ecco c and gamma',c,g)
            #print(cvacc)
            if(cvacc>maxacc):
                maxacc=cvacc
                maxg=g
                maxc=c
                bestsvm=rbf_svm
    
    print('ACCURACY on val:',maxacc)
    print('Gamma:',maxg)
    print('C:',maxc)
    #print(bestsvm)
    rbf_svm=svm.SVC(kernel='rbf',C=maxc,gamma=maxg)
    rbf_svm.fit(x_tvnew,y_tvnew)
    testacc=rbf_svm.score(x_testnew,y_testnew)
    print('Accuracy on TEST set:',testacc)
    accintime.append(testacc)
    
print(accintime)

[[0.         0.         0.         0.        ]
 [0.         1.         1.         1.        ]
 [0.         1.         1.         1.        ]
 ...
 [0.19658672 0.92926026 0.80571401 0.6867234 ]
 [0.30811808 0.84915922 0.78086245 0.67772984]
 [0.14297048 0.94312492 0.85122474 0.72747978]]
Working on window: 2
3517
Model Selection...
ACCURACY on val: 0.5285606478390851
Gamma: 1.0
C: 316.22776601683796
Accuracy on TEST set: 0.48525073746312686
Working on window: 6
3517
Model Selection...
ACCURACY on val: 0.5432987621278019
Gamma: 100.0
C: 10000.0
Accuracy on TEST set: 0.48668639053254437
Working on window: 12
3517
Model Selection...
ACCURACY on val: 0.5710648644782951
Gamma: 1.0
C: 10000.0
Accuracy on TEST set: 0.5379939209726444
Working on window: 30
3517
Model Selection...
ACCURACY on val: 0.5851749040848566
Gamma: 100.0
C: 10.0
Accuracy on TEST set: 0.4541139240506329
[0.48525073746312686, 0.48668639053254437, 0.5379939209726444, 0.4541139240506329]


In [3]:
============================================================
============================================================
Working on window: 2
3517
Model Selection...
ACCURACY on val: 0.5667283247960538
Gamma: 100.0
C: 316.22776601683796
Accuracy on TEST set: 0.47770700636942676
============================================================
============================================================
Working on window: 6
3517
Model Selection...
ACCURACY on val: 0.5620914682113995
Gamma: 100.0
C: 10000.0
Accuracy on TEST set: 0.5518394648829431
============================================================
============================================================
Working on window: 12
3517
Model Selection...
ACCURACY on val: 0.5853239109545163
Gamma: 100.0
C: 10.0
Accuracy on TEST set: 0.5471698113207547
============================================================
============================================================
Working on window: 30
3517
Model Selection...
ACCURACY on val: 0.5713741841914431
Gamma: 100.0
C: 10000.0
Accuracy on TEST set: 0.4957805907172996
[0.47770700636942676, 0.5518394648829431, 0.5471698113207547, 0.4957805907172996]

SyntaxError: invalid syntax (<ipython-input-3-436eb4b7de94>, line 1)